In [ ]:
import re, os
from wordcloud import WordCloud, STOPWORDS

In [ ]:
stopwords = STOPWORDS.copy()

In [ ]:
stopwords.add('said')
stopwords.add('will')

In [ ]:
path = '/home/subhasis/Dataset/Web Crawl data/bbc-fulltext/bbc/'

In [ ]:
directories = os.listdir(path)
directories

['entertainment', 'sport', 'business', 'tech', 'README.TXT', 'politics']

In [ ]:
directories.remove('README.TXT')
directories

['entertainment', 'sport', 'business', 'tech', 'politics']

In [ ]:
all_files = []
for directory in directories:
    file_path = path+directory+'/'
    files = os.listdir(file_path)
    for file in files:
        try:
            f= open(file_path+file).read()
            all_files.append((directory, f))
        except:
            continue

In [ ]:
all_news_df = pd.DataFrame.from_records(data=all_files, columns=['context','text'])
all_news_df.head()

,context,text
0,entertainment,Box office blow for Alexander\n\nDirector Oliv...
1,entertainment,Stars gear up for Bafta ceremony\n\nFilm stars...
2,entertainment,Stars shine on Bafta red carpet\n\nHollywood s...
3,entertainment,Aaliyah claim dismissed by court\n\nLate R&B s...
4,entertainment,Da Vinci Code is 'lousy history'\n\nThe plot o...


In [ ]:
all_news_df.shape

(2224, 2)

In [ ]:
only_texts = all_news_df['text']

In [ ]:
import re, nltk

In [ ]:
tokenizers = nltk.tokenize.RegexpTokenizer("[a-z']+")

In [ ]:
tokens_texts = [tokenizers.tokenize(t.lower()) for t in only_texts]

In [ ]:
import gensim

In [ ]:
tokens_texts_nostop = [[x for x in l if x not in STOPWORDS] for l in tokens_texts]

In [ ]:
from collections import Counter

In [ ]:
C = Counter([x for l in tokens_texts_nostop for x in l])

In [ ]:
vocab = [k for k,v in C.most_common(1000)]

In [ ]:
tokens_filtered = [[x for x in l if x in vocab] for l in tokens_texts_nostop]

In [ ]:
tokens_filtered[:20]

[['box',
  'office',
  'director',
  'failed',
  'bid',
  'box',
  'office',
  'us',
  'film',
  'number',
  'six',
  'cost',
  'm',
  'm',
  'make',
  'm',
  'three',
  'days',
  'us',
  'box',
  'office',
  'due',
  'film',
  'wednesday',
  'total',
  'm',
  'm',
  'top',
  'box',
  'office',
  'second',
  'week',
  'action',
  'movie',
  'national',
  'family',
  'took',
  'm',
  'm',
  'ahead',
  'comedy',
  'now',
  'fourth',
  'week',
  'took',
  'm',
  'm',
  'last',
  'week',
  'film',
  'met',
  'us',
  'critics',
  'film',
  'stars',
  'irish',
  'actor',
  'one',
  'much',
  'known',
  'world',
  'age',
  'met',
  'threat',
  'legal',
  'action',
  'though',
  'expect',
  'three',
  'hour',
  'one',
  'others',
  'called',
  'film',
  'saying',
  'last',
  'thursday',
  'award',
  'international',
  'film',
  'festival',
  'hope',
  'might',
  'better',
  'europe',
  'one',
  'see',
  'little',
  'us',
  'said',
  'director',
  'behind',
  'fourth',
  'added',
  'easy',
  'm

In [ ]:
id2word = gensim.corpora.Dictionary(tokens_texts)
corpus = [id2word.doc2bow(t) for t in tokens_filtered]

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=10, 
                                       random_state=123, passes=100, chunksize=500)

In [ ]:
print(gensim.models.CoherenceModel(lda_model, texts=tokens_texts, dictionary=id2word).get_coherence())

0.4927471199281002


In [ ]:
from hyperopt import tpe, hp, fmin, Trials, STATUS_OK

In [ ]:
def compute_coherence_values(params):
    
    lda_model = gensim.models.LdaMulticore(random_state=100, chunksize=500, passes=200, **params)
    
    coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=tokens_texts, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
params = {'alpha':hp.uniform('alpha',0.01,0.3),
        'eta':hp.uniform('eta',0.1,0.99),
        'num_topics':hp.uniformint('num_topics',2,10),
         'corpus':corpus,
         'id2word':id2word}

In [ ]:
def surrogate_function(params):
    perf = compute_coherence_values(params)
    return {'loss':-perf, 'status':STATUS_OK}

In [ ]:
trials = Trials()
best = fmin(surrogate_function, space=params, algo=tpe.suggest, max_evals=20, trials=trials)
print('best:')
print (best)

100%|██████████| 20/20 [41:26<00:00, 124.30s/trial, best loss: -0.5301218957626185]
best:
{'alpha': 0.1721780396972493, 'eta': 0.43801791695111414, 'num_topics': 3.0}


In [ ]:
lda_model_tuned = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=3, 
                                             random_state=100, passes=200, chunksize=500, 
                                             alpha=0.1721780396972493, eta=0.43801791695111414)

In [ ]:
import pyLDAvis.gensim_models# Visualize the topics
pyLDAvis.enable_notebook()

In [ ]:
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model_tuned, corpus, id2word)

LDAvis_prepared

/home/subhasis/miniconda3/envs/pytorch/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.157601  0.090635       1        1  70.189644
2     -0.057440 -0.123951       2        1  19.858503
1      0.215041  0.033316       3        1   9.951853, topic_info=             Term         Freq        Total Category  logprob  loglift
900            mr  2452.000000  2452.000000  Default  30.0000  30.0000
157          said  6335.000000  6335.000000  Default  29.0000  29.0000
481        people  1553.000000  1553.000000  Default  28.0000  28.0000
2303   government   833.000000   833.000000  Default  27.0000  27.0000
14402    election   525.000000   525.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
598         music   166.631558   704.648743   Topic3  -5.0391   0.8655
2285         used   149.546351   422.390388   Topic3  -5.1473   1.2691
628         video   136.602519   283.185978   Topic3  -5.2379   1.5784
895          many   144.922164   701.759968   Topic3  -5.1787   0.7300
900            mr   140.902451  2452.564906   Topic3  -5.2069  -0.5494

[211 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
4         1  0.996262   actor
9989      3  0.997339   apple
19        1  0.997379   award
218       1  0.996709  awards
643       1  0.896338    back
...     ...       ...     ...
551       1  0.974086    year
551       2  0.025375    year
351       1  0.997557  year's
552       1  0.928129   years
552       2  0.072407   years

[320 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

In [ ]:
only_texts[96]

'Veteran comic Cyril Fletcher dies\n\nVeteran comedian and broadcaster Cyril Fletcher has died aged 91 at his home in Guernsey.\n\nFletcher will be remembered for his "odd odes" and amusing misprints on TV show That\'s Life in the 1970s and 80s, as well as a long variety career. He was also a regular on Does the Team Think?, a comic version of radio information show The Brains Trust. That\'s Life host Esther Rantzen said he was "so lovely" and a "delight" to work with. "The thing about Cyril was that he was, to use a slightly old-fashioned phrase, an English gentleman," she said. "He was courteous, and understated, and he adored gardens, particularly creating them for his beautiful wife.\n\n"He was funny and witty to work with. He was a real friend and I shall miss him." One of the last comedians of old-time variety era, Fletcher made his first TV appearance in 1937. A year after BBC Television began, he was seen reciting humorous poems, appearing in a revue, Tele-Ho, and playing the E

In [ ]:
lda_model_tuned.get_document_topics(bow=corpus[96])

[(0, 0.99601126)]

In [ ]:
lda_model_tuned.get_topic_terms(0)

[(157, 0.024049496),
 (349, 0.018736428),
 (551, 0.0103713805),
 (193, 0.009793408),
 (141, 0.008348239),
 (466, 0.008177881),
 (117, 0.007950643),
 (111, 0.006634349),
 (884, 0.00654673),
 (753, 0.006217694)]

In [ ]:
word2id = {v:k for k,v in id2word.items()}

In [ ]:
[id2word[k] for k,_ in lda_model_tuned.get_topic_terms(0, 20)]

['said',
 'will',
 'year',
 'us',
 'one',
 'new',
 'm',
 'last',
 'first',
 'time',
 'two',
 'world',
 'best',
 'years',
 'now',
 'bn',
 'film',
 'game',
 'three',
 'back']